# Data Cleaning
Author: Marlin Figgins

In [1]:
# Importing neessary packages
import pandas as pd

In [2]:
# Loading Data
traffic_df = pd.read_csv("../data/traffic_df.csv")
field_interview_df = pd.read_csv("../data/field_interview_df.csv")

## For now, we'll restrict to traffic.
df = traffic_df
df.head()

,Date/Time,Location,Race,Gender,IDOT Classification,Reason for Stop,Citations/Violations,Disposition,Search
0,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015,No traffic stops for 06/01/2015
1,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015,No traffic stops for 06/02/2015
2,6/3/2015 9:26 AM,5600 S Stony Island,African American,Female,Traffic Sign/Signal,"Stop Sign Violation, Failed to Yield to Pedest...",NaN,Verbal Warning,No
3,6/4/2015 5:44 PM,5900 S Ellis,African American,Female,Follow Too Close,Following too closely to vehicle stopped for p...,NaN,Verbal Warning,No
4,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015,No traffic stops for 06/05/2015


In [3]:
# We must drop rows containing "No [X] for [Y]" 
df = df[~df["Date/Time"].str.contains("traffic stops|field interviews|no|No")].reset_index(drop = True)
df.head()

,Date/Time,Location,Race,Gender,IDOT Classification,Reason for Stop,Citations/Violations,Disposition,Search
0,6/3/2015 9:26 AM,5600 S Stony Island,African American,Female,Traffic Sign/Signal,"Stop Sign Violation, Failed to Yield to Pedest...",NaN,Verbal Warning,No
1,6/4/2015 5:44 PM,5900 S Ellis,African American,Female,Follow Too Close,Following too closely to vehicle stopped for p...,NaN,Verbal Warning,No
2,6/27/2015 11:36 AM,6121 S Cottage Grove,African American,Male,Lane Violation,Subject was driving erratically and cut off an...,NaN,Verbal Warning,No
3,7/11/2015 10:04 PM,5500 S Cottage Grove,African American,Male,Moving Violation - Other,Failed to yield to CFD firetruck,NaN,Verbal Warning,No
4,7/11/2015 10:47 PM,5600 S Drexel,African American,Male,Lane Violation,Wrong way on one way road,NaN,Verbal Warning,No


# Standardizing and cleanig "Race" column

In [4]:
df['Race'].value_counts()

African American                          3276
Caucasian                                  723
Hispanic                                   217
Asian                                      208
ASIAN                                       18
CAUCASIAN                                   17
American Indian/Alaskan Native              10
Native Hawaiian/Other Pacific Islander       4
American Indian                              2
Female                                       1
African                                      1
Male                                         1
Name: Race, dtype: int64

Notice that there are issues with capitalization in the cases of "ASIAN", "CAUCASIAN", and issues with missing "/" and incomplete terms. 

We'll use the five racial categories used in the 2000 census as our standard for cleaning here:
-  White American
- Black or African American
- Native American or Alaska Native:
- Asian American
- Native Hawaiian or Other Pacific Islander

This means that the general issues in our data we're seeking to resolve are as follows:

- If an entry contains "African", we'll classify as "Black / African American"
- If an entry contains "American Indian", we'll classify as "American Indian/Alaskan Native"  
- Standard all entries to be in capital letters.

In [5]:
def standardize_race(race):
    """
    Takes in "Race" entry and returns in standardized format.
    """
    race = str(race).upper() # force upper case

    if "AMERICAN INDIAN" in race or "ALASKA" in race or "NATIVE AMERICAN" in race:
        return "NATIVE AMERICAN / ALASKAN NATIVE"

    if "AFRICAN" in race or "BLACK" in race:
        return "BLACK / AFRICAN AMERICAN"

    if "CAUCASIAN" in race or "WHITE" in race:
        return "WHITE AMERICAN / CAUCASIAN"

    if "HAWAIIAN" in race or "PACIFIC" in race:
        return "NATIVE HAWAIIAN / OTHER PACIFIC ISLANDER"

    if "HISPANIC" in race:
        return "HISPANIC"
    
    # If unclassified, return unchanged
    return race

In [6]:
df['Race'] = df["Race"].apply(standardize_race)
df['Race'].value_counts()

BLACK / AFRICAN AMERICAN                    3277
WHITE AMERICAN / CAUCASIAN                   740
ASIAN                                        226
HISPANIC                                     217
NATIVE AMERICAN / ALASKAN NATIVE              12
NATIVE HAWAIIAN / OTHER PACIFIC ISLANDER       4
MALE                                           1
FEMALE                                         1
Name: Race, dtype: int64

There's still the issue of the MALE and FEMALE entries but this appears to be due to a shifted row, but we'll drop them for ease since it is only two data points.

In [7]:
# REMOVE ROW IF GENDER IN RACE COLUMN
df[~df["Race"].isin(["MALE", "FEMALE"])]

,Date/Time,Location,Race,Gender,IDOT Classification,Reason for Stop,Citations/Violations,Disposition,Search
0,6/3/2015 9:26 AM,5600 S Stony Island,BLACK / AFRICAN AMERICAN,Female,Traffic Sign/Signal,"Stop Sign Violation, Failed to Yield to Pedest...",NaN,Verbal Warning,No
1,6/4/2015 5:44 PM,5900 S Ellis,BLACK / AFRICAN AMERICAN,Female,Follow Too Close,Following too closely to vehicle stopped for p...,NaN,Verbal Warning,No
2,6/27/2015 11:36 AM,6121 S Cottage Grove,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Subject was driving erratically and cut off an...,NaN,Verbal Warning,No
3,7/11/2015 10:04 PM,5500 S Cottage Grove,BLACK / AFRICAN AMERICAN,Male,Moving Violation - Other,Failed to yield to CFD firetruck,NaN,Verbal Warning,No
4,7/11/2015 10:47 PM,5600 S Drexel,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Wrong way on one way road,NaN,Verbal Warning,No
...,...,...,...,...,...,...,...,...,...
4473,8/10/2020 12:44 PM,6200 S. Greenwood,BLACK / AFRICAN AMERICAN,Male,Traffic sign/signal,Disobey stop sign and failure to give right of...,NaN,Verbal Warning,NO
4474,8/11/2020 9:06 PM,5300 S. University,BLACK / AFRICAN AMERICAN,Male,Traffic sign/signal,Disobeyed Stop Sign,NaN,Verbal Warning,NO
4475,8/11/2020 11:05 PM,1412 E. 53rd Street,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Wrong way on one way street,One Citation Issued,Citation Issued,NO
4476,8/12/2020 4:56 PM,1455 E. 53rd Street,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,NO


# Separating Dates

In [8]:
df.head()

,Date/Time,Location,Race,Gender,IDOT Classification,Reason for Stop,Citations/Violations,Disposition,Search
0,6/3/2015 9:26 AM,5600 S Stony Island,BLACK / AFRICAN AMERICAN,Female,Traffic Sign/Signal,"Stop Sign Violation, Failed to Yield to Pedest...",NaN,Verbal Warning,No
1,6/4/2015 5:44 PM,5900 S Ellis,BLACK / AFRICAN AMERICAN,Female,Follow Too Close,Following too closely to vehicle stopped for p...,NaN,Verbal Warning,No
2,6/27/2015 11:36 AM,6121 S Cottage Grove,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Subject was driving erratically and cut off an...,NaN,Verbal Warning,No
3,7/11/2015 10:04 PM,5500 S Cottage Grove,BLACK / AFRICAN AMERICAN,Male,Moving Violation - Other,Failed to yield to CFD firetruck,NaN,Verbal Warning,No
4,7/11/2015 10:47 PM,5600 S Drexel,BLACK / AFRICAN AMERICAN,Male,Lane Violation,Wrong way on one way road,NaN,Verbal Warning,No


Since there are formatting inconsisenticies in Date/Time formatting in this dataset, we'll split Date and Time into seperate columns.

In [9]:
# Splitting Date Time
df['Date'] = df["Date/Time"].apply(lambda x: x.split()[0])
df['Time'] = df["Date/Time"].apply(lambda x: x.split(" ", 1)[1])

# Standardizing Gender column

In [10]:
df['Gender'] = df.Gender.str.upper()
df['Gender'].value_counts()

MALE                                  2860
FEMALE                                1615
FAILURE TO YIELD TO CROSSING GUARD       2
Name: Gender, dtype: int64

In [11]:
## Script to implement

def clean_UCPD_data(in_df):
    """
    This cleans the raw data returned from scrape_UCPD_data.
    Removes shifted rows, parses dates, and standardizes Gender and Race.
    """
    # We must drop rows containing "No [X] for [Y]" 
    df = in_df[~in_df["Date/Time"].str.contains("traffic stops|field interviews|no|No")].reset_index(drop = True)

    # Standardizes gender capitalization
    df['Gender'] = df.Gender.str.upper()

    # Imposes 2000 census racial categoriees
    df['Race'] = df["Race"].apply(standardize_race)

    # REMOVE IF GENDER IN RACE COLUMN
    df[~df["Race"].isin(["MALE", "FEMALE"])]

    # Splits into Date, Time columns separately
    df['Date'] = df["Date/Time"].apply(lambda x: x.split(" ", 1)[0])
    df['Time'] = df["Date/Time"].apply(lambda x: x.split(" ", 1)[1])
    
    # Drop "Date/Time"
    df = df.drop(columns = ["Date/Time"])
    return df

In [12]:
import time

start = time.time()
field_interview_df_cleaned = clean_UCPD_data(field_interview_df)
field_interview_df_cleaned.to_csv("../data/field_interview_df_cleaned.csv", index = False)
end = time.time()
print(f"Cleaning data and exporting field_interview_df_cleaned.csv took {end - start} seconds.")
field_interview_df_cleaned.head()

Cleaning data and exporting field_interview_df_cleaned.csv took 0.01572108268737793 seconds.


,Location,Initiated By,Race,Gender,Reason for Stop,Disposition,Search,Date,Time
0,1601 E 53rd,Citizen request for UCPD Response,BLACK / AFRICAN AMERICAN,FEMALE,Citizen observed subject having a verbal argum...,Name checked; no further action,No,6/3/2015,1:40 PM
1,1601 E 53rd,Citizen request for UCPD Response,BLACK / AFRICAN AMERICAN,MALE,Citizen observed subject having a verbal argum...,Name checked; no further action,No,6/3/2015,1:40 PM
2,5245 S Cottage Grove,Citizen request for UCPD Response,BLACK / AFRICAN AMERICAN,MALE,Complainant advised subject acted suspicious (...,Name checked; no further action,No,6/4/2015,8:21 PM
3,901 E 58th Street,UCPD,BLACK / AFRICAN AMERICAN,MALE,"Disturbance - Domestic Incident, prior inciden...",Name checked; no further action,No,6/4/2015,10:01 PM
4,901 E 58th Street,UCPD,BLACK / AFRICAN AMERICAN,FEMALE,"Disturbance - Domestic Incident, prior inciden...",Name checked; arrested by CPD,No,6/4/2015,10:01 PM


In [13]:
start = time.time()
traffic_df_cleaned = clean_UCPD_data(traffic_df)
traffic_df_cleaned.to_csv("../data/traffic_df_cleaned.csv", index = False)
end = time.time()
print(f"Cleaning data and exporting traffic_df_cleaned.csv took {end - start} seconds.")
traffic_df_cleaned.head()

Cleaning data and exporting traffic_df_cleaned.csv took 0.041895151138305664 seconds.


,Location,Race,Gender,IDOT Classification,Reason for Stop,Citations/Violations,Disposition,Search,Date,Time
0,5600 S Stony Island,BLACK / AFRICAN AMERICAN,FEMALE,Traffic Sign/Signal,"Stop Sign Violation, Failed to Yield to Pedest...",NaN,Verbal Warning,No,6/3/2015,9:26 AM
1,5900 S Ellis,BLACK / AFRICAN AMERICAN,FEMALE,Follow Too Close,Following too closely to vehicle stopped for p...,NaN,Verbal Warning,No,6/4/2015,5:44 PM
2,6121 S Cottage Grove,BLACK / AFRICAN AMERICAN,MALE,Lane Violation,Subject was driving erratically and cut off an...,NaN,Verbal Warning,No,6/27/2015,11:36 AM
3,5500 S Cottage Grove,BLACK / AFRICAN AMERICAN,MALE,Moving Violation - Other,Failed to yield to CFD firetruck,NaN,Verbal Warning,No,7/11/2015,10:04 PM
4,5600 S Drexel,BLACK / AFRICAN AMERICAN,MALE,Lane Violation,Wrong way on one way road,NaN,Verbal Warning,No,7/11/2015,10:47 PM
